# Reserve Data Exploration
Extract and examine ENTSOE reserve price data from ZIP files


In [1]:
import requests
import zipfile
import io
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
from datetime import datetime


In [2]:
# Helper function to fetch data in chunks (avoid API limits)
def fetch_entsoe_in_chunks(base_url, start_date, end_date, chunk_days=15):
    """
    Fetch ENTSOE data in chunks to avoid API limits.
    
    Args:
        base_url: URL without periodStart/periodEnd parameters
        start_date: Start date string "YYYYMMDD0000"
        end_date: End date string "YYYYMMDD0000"
        chunk_days: Number of days per request (default 15)
    
    Returns:
        Combined response content (ZIP or XML)
    """
    from datetime import datetime, timedelta
    import time
    
    # Parse dates
    start = datetime.strptime(start_date, "%Y%m%d%H%M")
    end = datetime.strptime(end_date, "%Y%m%d%H%M")
    
    all_zips = []
    current = start
    
    while current < end:
        next_chunk = min(current + timedelta(days=chunk_days), end)
        
        period_start = current.strftime("%Y%m%d%H%M")
        period_end = next_chunk.strftime("%Y%m%d%H%M")
        
        url = f"{base_url}&periodStart={period_start}&periodEnd={period_end}"
        
        print(f"Fetching {period_start} to {period_end}...")
        
        response = requests.get(url)
        
        if response.status_code == 200:
            all_zips.append(response.content)
            print(f"  ✓ Got {len(response.content)} bytes")
        else:
            print(f"  ✗ Error {response.status_code}: {response.text[:200]}")
        
        current = next_chunk
        time.sleep(0.5)  # Rate limiting
    
    # Combine all ZIP files
    if len(all_zips) == 1:
        return all_zips[0]
    
    # Merge multiple ZIPs into one
    combined_buf = io.BytesIO()
    with zipfile.ZipFile(combined_buf, 'w', compression=zipfile.ZIP_DEFLATED) as zout:
        for idx, zip_content in enumerate(all_zips):
            try:
                ztemp = zipfile.ZipFile(io.BytesIO(zip_content))
                for name in ztemp.namelist():
                    data = ztemp.read(name)
                    out_name = f"chunk{idx}_{name}"
                    zout.writestr(out_name, data)
            except zipfile.BadZipFile:
                # Might be plain XML, wrap it
                zout.writestr(f"chunk{idx}_response.xml", zip_content)
    
    combined_buf.seek(0)
    return combined_buf.read()

print("Helper function loaded")


Helper function loaded


In [3]:
# aFRR Daily capacity (ProcessType=A51) — Fetch September 2024 only (simplified)

API_KEY = "3da11da4-cb1c-41ea-8f2e-e80e188b9e4b"  # reuse or replace with your token

# September 2024
start, end = "202409010000", "202410010000"

# Base URL (Hungary control area)
base_url = (
    "https://web-api.tp.entsoe.eu/api?"
    "documentType=A81&"
    "businessType=B95&"
    "processType=A51&"
    "Type_MarketAgreement.Type=A01&"
    "controlArea_Domain=10YHU-MAVIR----U&"
    f"securityToken={API_KEY}"
)

print(f"Fetching aFRR Daily data for September 2024: {start[:8]} to {end[:8]}")

# If the helper exists in this notebook, use it (keeps retry/chunk logic); otherwise do a single GET
try:
    fetch_entsoe_in_chunks
    content = fetch_entsoe_in_chunks(base_url, start, end, chunk_days=7)
except NameError:
    import requests
    params = {
        "documentType": "A81",
        "businessType": "B95",
        "processType": "A51",
        "Type_MarketAgreement.Type": "A01",
        "controlArea_Domain": "10YHU-MAVIR----U",
        "securityToken": API_KEY,
        "periodStart": start,
        "periodEnd": end,
    }
    r = requests.get("https://web-api.tp.entsoe.eu/api", params=params, timeout=60)
    r.raise_for_status()
    content = r.content

# Minimal shim to keep downstream cells working: response_afrr_d1.content
class MockResponse:
    def __init__(self, content):
        self.content = content
        self.status_code = 200

response_afrr_d1 = MockResponse(content)
print(f"✓ Total fetched bytes for September: {len(content)}")



Fetching aFRR Daily data for September 2024: 20240901 to 20241001
Fetching 202409010000 to 202409080000...
  ✓ Got 8816 bytes
Fetching 202409080000 to 202409150000...
  ✓ Got 8702 bytes
Fetching 202409150000 to 202409220000...
  ✓ Got 9080 bytes
Fetching 202409220000 to 202409290000...
  ✓ Got 9429 bytes
Fetching 202409290000 to 202410010000...
  ✓ Got 4334 bytes
✓ Total fetched bytes for September: 40263


In [4]:
responses = [response_afrr_d1]

In [5]:
# Extract ZIP file

import io, zipfile
from pathlib import Path


zip_file1 = zipfile.ZipFile(io.BytesIO(response_afrr_d1.content))



zips = [zip_file1]  # your ZipFile objects

combined_buf = io.BytesIO()
with zipfile.ZipFile(combined_buf, 'w', compression=zipfile.ZIP_DEFLATED) as zout:
    for i, z in enumerate(zips):
        for name in z.namelist():
            data = z.read(name)
            out_name = f"{i}_{Path(name).name}"  # avoid name collisions
            zout.writestr(out_name, data)

combined_buf.seek(0)
zf = zipfile.ZipFile(combined_buf, 'r')   # merged zip
xml_files = zf.namelist()                 # flat list of all entries
print(f"Merged {len(xml_files)} XML files")

xml_files_afrr_d1 = zip_file1.namelist()                 # flat list of all entries




Merged 5 XML files


In [6]:
# Extract XML files into a single DataFrame (store raw XML text)
import re


rows = []

for xml_file in xml_files:
    print(f"\nProcessing {xml_file}...")

    # Read XML content and decode (handle BOM safely)
    xml_bytes = zf.read(xml_file)
    xml_text = xml_bytes.decode('utf-8-sig', errors='replace')

    # Split concatenated XML documents while preserving the declaration
    docs = [d for d in re.split(r'(?=<\?xml\s)', xml_text) if d.strip()]
    if not docs:
        docs = [xml_text]

    for i, doc in enumerate(docs):
        # Ensure declaration at start; if missing, add a minimal one
        cleaned = doc.lstrip()
        if not cleaned.startswith('<?xml'):
            cleaned = '<?xml version="1.0" encoding="UTF-8"?>\n' + cleaned

        name = f"{xml_file}_part_{i}" if len(docs) > 1 else xml_file
        rows.append({'file': name, 'xml': cleaned})

xml_docs_df = pd.DataFrame(rows)
print(f"Collected {len(xml_docs_df)} XML documents into xml_docs_df")
display(xml_docs_df.head())



Processing 0_chunk0_001-AMOUNT_AND_PRICES_PAID_OF_BALANCING_RESERVES_UNDER_CONTRACT_R3_202408312200-202409082200.xml...

Processing 0_chunk1_001-AMOUNT_AND_PRICES_PAID_OF_BALANCING_RESERVES_UNDER_CONTRACT_R3_202409072200-202409152200.xml...

Processing 0_chunk2_001-AMOUNT_AND_PRICES_PAID_OF_BALANCING_RESERVES_UNDER_CONTRACT_R3_202409142200-202409222200.xml...

Processing 0_chunk3_001-AMOUNT_AND_PRICES_PAID_OF_BALANCING_RESERVES_UNDER_CONTRACT_R3_202409212200-202409292200.xml...

Processing 0_chunk4_001-AMOUNT_AND_PRICES_PAID_OF_BALANCING_RESERVES_UNDER_CONTRACT_R3_202409282200-202410012200.xml...
Collected 5 XML documents into xml_docs_df


,file,xml
0,0_chunk0_001-AMOUNT_AND_PRICES_PAID_OF_BALANCI...,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<Balan..."
1,0_chunk1_001-AMOUNT_AND_PRICES_PAID_OF_BALANCI...,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<Balan..."
2,0_chunk2_001-AMOUNT_AND_PRICES_PAID_OF_BALANCI...,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<Balan..."
3,0_chunk3_001-AMOUNT_AND_PRICES_PAID_OF_BALANCI...,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<Balan..."
4,0_chunk4_001-AMOUNT_AND_PRICES_PAID_OF_BALANCI...,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<Balan..."


In [7]:
from lxml import etree as LET
print("Imported LET from lxml.etree")


Imported LET from lxml.etree


In [8]:
# Parse a single balancing XML into one DataFrame (metadata + all points)
from lxml import etree as LET
from pathlib import Path
import pandas as pd


def _lname(tag: str) -> str:
    return tag.split('}')[-1] if '}' in tag else tag


def parse_balancing_xml_to_df(xml_text: str) -> pd.DataFrame:
    root = LET.fromstring(xml_text.encode('utf-8'))

    # Document-level metadata (leaf nodes outside Point)
    meta = {}
    for el in root.iter():
        ln = _lname(el.tag)
        if ln == 'Point':
            continue
        # skip if any Point descendant
        if el.xpath('.//*[local-name()="Point"]'):
            continue
        if len(list(el)) == 0:
            val = (el.text or '').strip()
            if val:
                key = f"doc.{ln}"
                # keep first occurrence
                if key not in meta:
                    meta[key] = val
    # include root attributes
    for k, v in root.attrib.items():
        meta[f"doc.@{k}"] = v

    rows = []
    # Iterate TimeSeries → Period → Point
    for ts in root.xpath('.//*[local-name()="TimeSeries"]'):
        ts_meta = {}
        for child in ts:
            ln = _lname(child.tag)
            if ln in ('Period', 'timeInterval'):
                continue
            # capture simple leaf text under TS (one level)
            if len(list(child)) == 0:
                ts_meta[f"ts.{ln}"] = (child.text or '').strip()
        for per in ts.xpath('.//*[local-name()="Period"]'):
            # period context
            start = per.xpath('.//*[local-name()="timeInterval"]/*[local-name()="start"]/text()')
            end = per.xpath('.//*[local-name()="timeInterval"]/*[local-name()="end"]/text()')
            res = per.xpath('.//*[local-name()="resolution"]/text()')
            per_meta = {
                'per.start': start[0] if start else None,
                'per.end': end[0] if end else None,
                'per.resolution': res[0] if res else None,
            }
            for pt in per.xpath('.//*[local-name()="Point"]'):
                row = {}
                # Point fields (flatten one level deep)
                for ch in list(pt):
                    ln = _lname(ch.tag)
                    if len(list(ch)) == 0:
                        row[f"pt.{ln}"] = (ch.text or '').strip()
                    else:
                        # flatten nested elements under Point one level
                        for sub in ch:
                            row[f"pt.{ln}.{_lname(sub.tag)}"] = (sub.text or '').strip()
                # Merge: document meta + TS meta + period meta + point
                full = {**meta, **ts_meta, **per_meta, **row}
                rows.append(full)
    return pd.DataFrame(rows)

# Source XML: prefer first entry in zf/xml_files, else fallback to example file
if 'zf' in globals() and 'xml_files' in globals() and xml_files:
    xml_text_src = zf.read(xml_files[0]).decode('utf-8-sig', errors='replace')
else:
    example_path = Path('notebooks/ActivationPriceXML.txt')
    if not example_path.exists():
        example_path = Path('../notebooks/ActivationPriceXML.txt')
    xml_text_src = example_path.read_text(encoding='utf-8')

points_full_df = parse_balancing_xml_to_df(xml_text_src)
print(points_full_df.shape)
display(points_full_df.head())


(929, 40)


,doc.mRID,doc.revisionNumber,doc.type,doc.process.processType,doc.sender_MarketParticipant.mRID,doc.sender_MarketParticipant.marketRole.type,doc.receiver_MarketParticipant.mRID,doc.receiver_MarketParticipant.marketRole.type,doc.createdDateTime,doc.area_Domain.mRID,...,ts.flowDirection.direction,ts.currency_Unit.name,ts.quantity_Measure_Unit.name,ts.curveType,per.start,per.end,per.resolution,pt.position,pt.quantity,pt.procurement_Price.amount
0,19eeabfbd0d0416f80dd81c0b7121265,1,A81,A51,10X1001A1001A450,A32,10X1001A1001A450,A33,2025-11-07T13:30:19Z,10YHU-MAVIR----U,...,A01,HUF,MAW,A03,2024-08-31T22:00Z,2024-09-01T22:00Z,PT15M,1,70,76
1,19eeabfbd0d0416f80dd81c0b7121265,1,A81,A51,10X1001A1001A450,A32,10X1001A1001A450,A33,2025-11-07T13:30:19Z,10YHU-MAVIR----U,...,A01,HUF,MAW,A03,2024-08-31T22:00Z,2024-09-01T22:00Z,PT15M,21,70,77
2,19eeabfbd0d0416f80dd81c0b7121265,1,A81,A51,10X1001A1001A450,A32,10X1001A1001A450,A33,2025-11-07T13:30:19Z,10YHU-MAVIR----U,...,A01,HUF,MAW,A03,2024-08-31T22:00Z,2024-09-01T22:00Z,PT15M,25,70,82
3,19eeabfbd0d0416f80dd81c0b7121265,1,A81,A51,10X1001A1001A450,A32,10X1001A1001A450,A33,2025-11-07T13:30:19Z,10YHU-MAVIR----U,...,A01,HUF,MAW,A03,2024-08-31T22:00Z,2024-09-01T22:00Z,PT15M,29,35,6250
4,19eeabfbd0d0416f80dd81c0b7121265,1,A81,A51,10X1001A1001A450,A32,10X1001A1001A450,A33,2025-11-07T13:30:19Z,10YHU-MAVIR----U,...,A01,HUF,MAW,A03,2024-08-31T22:00Z,2024-09-01T22:00Z,PT15M,33,31,8495


In [9]:
# Parse all XML documents in xml_docs_df into one DataFrame (metadata + points)
import pandas as pd

if 'xml_docs_df' not in globals() or xml_docs_df.empty:
    raise RuntimeError('xml_docs_df is empty or not defined. Build it first.')

all_rows = []
for _, r in xml_docs_df.iterrows():
    try:
        dfi = parse_balancing_xml_to_df(str(r['xml']))
        dfi['file'] = r.get('file', None)
        all_rows.append(dfi)
    except Exception as e:
        print(f"Skip {r.get('file', '?')}: {e}")

points_full_df = pd.concat(all_rows, ignore_index=True) if all_rows else pd.DataFrame()
print(f"points_full_df combined: shape={points_full_df.shape}, columns={len(points_full_df.columns)}")
display(points_full_df.head())


points_full_df combined: shape=(4175, 41), columns=41


,doc.mRID,doc.revisionNumber,doc.type,doc.process.processType,doc.sender_MarketParticipant.mRID,doc.sender_MarketParticipant.marketRole.type,doc.receiver_MarketParticipant.mRID,doc.receiver_MarketParticipant.marketRole.type,doc.createdDateTime,doc.area_Domain.mRID,...,ts.currency_Unit.name,ts.quantity_Measure_Unit.name,ts.curveType,per.start,per.end,per.resolution,pt.position,pt.quantity,pt.procurement_Price.amount,file
0,19eeabfbd0d0416f80dd81c0b7121265,1,A81,A51,10X1001A1001A450,A32,10X1001A1001A450,A33,2025-11-07T13:30:19Z,10YHU-MAVIR----U,...,HUF,MAW,A03,2024-08-31T22:00Z,2024-09-01T22:00Z,PT15M,1,70,76,0_chunk0_001-AMOUNT_AND_PRICES_PAID_OF_BALANCI...
1,19eeabfbd0d0416f80dd81c0b7121265,1,A81,A51,10X1001A1001A450,A32,10X1001A1001A450,A33,2025-11-07T13:30:19Z,10YHU-MAVIR----U,...,HUF,MAW,A03,2024-08-31T22:00Z,2024-09-01T22:00Z,PT15M,21,70,77,0_chunk0_001-AMOUNT_AND_PRICES_PAID_OF_BALANCI...
2,19eeabfbd0d0416f80dd81c0b7121265,1,A81,A51,10X1001A1001A450,A32,10X1001A1001A450,A33,2025-11-07T13:30:19Z,10YHU-MAVIR----U,...,HUF,MAW,A03,2024-08-31T22:00Z,2024-09-01T22:00Z,PT15M,25,70,82,0_chunk0_001-AMOUNT_AND_PRICES_PAID_OF_BALANCI...
3,19eeabfbd0d0416f80dd81c0b7121265,1,A81,A51,10X1001A1001A450,A32,10X1001A1001A450,A33,2025-11-07T13:30:19Z,10YHU-MAVIR----U,...,HUF,MAW,A03,2024-08-31T22:00Z,2024-09-01T22:00Z,PT15M,29,35,6250,0_chunk0_001-AMOUNT_AND_PRICES_PAID_OF_BALANCI...
4,19eeabfbd0d0416f80dd81c0b7121265,1,A81,A51,10X1001A1001A450,A32,10X1001A1001A450,A33,2025-11-07T13:30:19Z,10YHU-MAVIR----U,...,HUF,MAW,A03,2024-08-31T22:00Z,2024-09-01T22:00Z,PT15M,33,31,8495,0_chunk0_001-AMOUNT_AND_PRICES_PAID_OF_BALANCI...


In [10]:
points_full_df.columns

Index(['doc.mRID', 'doc.revisionNumber', 'doc.type', 'doc.process.processType',
       'doc.sender_MarketParticipant.mRID',
       'doc.sender_MarketParticipant.marketRole.type',
       'doc.receiver_MarketParticipant.mRID',
       'doc.receiver_MarketParticipant.marketRole.type', 'doc.createdDateTime',
       'doc.area_Domain.mRID', 'doc.allocationDecision_DateAndOrTime.dateTime',
       'doc.start', 'doc.end', 'doc.businessType',
       'doc.type_MarketAgreement.type',
       'doc.original_MarketProduct.marketProductType',
       'doc.mktPSRType.psrType', 'doc.flowDirection.direction',
       'doc.currency_Unit.name', 'doc.quantity_Measure_Unit.name',
       'doc.curveType', 'doc.resolution', 'doc.position', 'doc.quantity',
       'doc.procurement_Price.amount', 'ts.mRID', 'ts.businessType',
       'ts.type_MarketAgreement.type',
       'ts.original_MarketProduct.marketProductType', 'ts.mktPSRType.psrType',
       'ts.flowDirection.direction', 'ts.currency_Unit.name',
       'ts.quan

In [11]:
unique_counts = points_full_df.nunique(dropna=True).sort_values(ascending=False).to_frame("n_unique")
print(f"DataFrame shape: {unique_counts.shape}")
display(unique_counts)


DataFrame shape: (41, 1)


,n_unique
pt.procurement_Price.amount,2211
pt.quantity,166
ts.mRID,65
per.start,31
per.end,31
pt.position,24
doc.end,5
doc.createdDateTime,5
doc.mRID,5
doc.procurement_Price.amount,5


In [12]:
points_full_df =  points_full_df[[
    "doc.process.processType",
    "ts.type_MarketAgreement.type",
    "ts.mktPSRType.psrType",
    "ts.flowDirection.direction",
    "per.start",
    "per.end",
    "per.resolution",
    "pt.position",
    "pt.quantity",
    "pt.procurement_Price.amount",
    "ts.currency_Unit.name",
]]


In [13]:
points_full_df

,doc.process.processType,ts.type_MarketAgreement.type,ts.mktPSRType.psrType,ts.flowDirection.direction,per.start,per.end,per.resolution,pt.position,pt.quantity,pt.procurement_Price.amount,ts.currency_Unit.name
0,A51,A01,A03,A01,2024-08-31T22:00Z,2024-09-01T22:00Z,PT15M,1,70,76,HUF
1,A51,A01,A03,A01,2024-08-31T22:00Z,2024-09-01T22:00Z,PT15M,21,70,77,HUF
2,A51,A01,A03,A01,2024-08-31T22:00Z,2024-09-01T22:00Z,PT15M,25,70,82,HUF
3,A51,A01,A03,A01,2024-08-31T22:00Z,2024-09-01T22:00Z,PT15M,29,35,6250,HUF
4,A51,A01,A03,A01,2024-08-31T22:00Z,2024-09-01T22:00Z,PT15M,33,31,8495,HUF
...,...,...,...,...,...,...,...,...,...,...,...
4170,A51,A01,A05,A02,2024-09-30T22:00Z,2024-10-01T22:00Z,PT15M,77,21,11014,HUF
4171,A51,A01,A05,A02,2024-09-30T22:00Z,2024-10-01T22:00Z,PT15M,81,21,605,HUF
4172,A51,A01,A05,A02,2024-09-30T22:00Z,2024-10-01T22:00Z,PT15M,85,21,799,HUF
4173,A51,A01,A05,A02,2024-09-30T22:00Z,2024-10-01T22:00Z,PT15M,89,20,414,HUF


In [14]:
# parse UTC 'Z' strings
start_utc = pd.to_datetime(points_full_df['per.start'], utc=True, errors='coerce')
end_utc   = pd.to_datetime(points_full_df['per.end'],   utc=True, errors='coerce')

# DST-aware Hungary time (Europe/Budapest)
points_full_df['per.start_dt'] = start_utc.dt.tz_convert('Europe/Budapest')
points_full_df['per.end_dt']   = end_utc.dt.tz_convert('Europe/Budapest')

points_full_df['per.start_dt'] = start_utc.dt.tz_convert('Etc/GMT-2')
points_full_df['per.end_dt']   = end_utc.dt.tz_convert('Etc/GMT-2')

points_full_df.head()

,doc.process.processType,ts.type_MarketAgreement.type,ts.mktPSRType.psrType,ts.flowDirection.direction,per.start,per.end,per.resolution,pt.position,pt.quantity,pt.procurement_Price.amount,ts.currency_Unit.name,per.start_dt,per.end_dt
0,A51,A01,A03,A01,2024-08-31T22:00Z,2024-09-01T22:00Z,PT15M,1,70,76,HUF,2024-09-01 00:00:00+02:00,2024-09-02 00:00:00+02:00
1,A51,A01,A03,A01,2024-08-31T22:00Z,2024-09-01T22:00Z,PT15M,21,70,77,HUF,2024-09-01 00:00:00+02:00,2024-09-02 00:00:00+02:00
2,A51,A01,A03,A01,2024-08-31T22:00Z,2024-09-01T22:00Z,PT15M,25,70,82,HUF,2024-09-01 00:00:00+02:00,2024-09-02 00:00:00+02:00
3,A51,A01,A03,A01,2024-08-31T22:00Z,2024-09-01T22:00Z,PT15M,29,35,6250,HUF,2024-09-01 00:00:00+02:00,2024-09-02 00:00:00+02:00
4,A51,A01,A03,A01,2024-08-31T22:00Z,2024-09-01T22:00Z,PT15M,33,31,8495,HUF,2024-09-01 00:00:00+02:00,2024-09-02 00:00:00+02:00


In [15]:
# Start of the day-period in Europe/Budapest
start_dt = (
    pd.to_datetime(points_full_df['per.start'], utc=True, errors='coerce')
      .dt.tz_convert('Europe/Budapest')
)

# Parse ISO 8601 resolution (PT15M/PT30M/PT60M/PT1H...) -> minutes
res_ex = points_full_df['per.resolution'].str.upper().str.extract(r'PT(\d+)([HM])')
mins = pd.to_numeric(res_ex[0], errors='coerce')
mins = np.where(res_ex[1].eq('H'), mins * 60, mins)
mins = pd.to_numeric(mins, errors='coerce')

# Positions are 1-based; compute offset and exact time
pos = pd.to_numeric(points_full_df['pt.position'], errors='coerce')
offset_min = (pos - 1) * mins

points_full_df['time_dt'] = start_dt + pd.to_timedelta(offset_min, unit='m')
points_full_df['time_end_dt'] = points_full_df['time_dt'] + pd.to_timedelta(mins, unit='m')

points_full_df[['per.start','per.resolution','pt.position','time_dt','time_end_dt']].head()

,per.start,per.resolution,pt.position,time_dt,time_end_dt
0,2024-08-31T22:00Z,PT15M,1,2024-09-01 00:00:00+02:00,2024-09-01 00:15:00+02:00
1,2024-08-31T22:00Z,PT15M,21,2024-09-01 05:00:00+02:00,2024-09-01 05:15:00+02:00
2,2024-08-31T22:00Z,PT15M,25,2024-09-01 06:00:00+02:00,2024-09-01 06:15:00+02:00
3,2024-08-31T22:00Z,PT15M,29,2024-09-01 07:00:00+02:00,2024-09-01 07:15:00+02:00
4,2024-08-31T22:00Z,PT15M,33,2024-09-01 08:00:00+02:00,2024-09-01 08:15:00+02:00


In [16]:
points_full_df =  points_full_df[[
    'per.start_dt', 
    'per.end_dt' ,
    'time_dt',
    'time_end_dt',
    'per.resolution',
    'pt.position', 
    "doc.process.processType",
    "ts.type_MarketAgreement.type",
    "ts.mktPSRType.psrType",
    "ts.flowDirection.direction",
    "pt.quantity",
    "pt.procurement_Price.amount",
    "ts.currency_Unit.name",
]]

In [17]:
points_full_df.head()

,per.start_dt,per.end_dt,time_dt,time_end_dt,per.resolution,pt.position,doc.process.processType,ts.type_MarketAgreement.type,ts.mktPSRType.psrType,ts.flowDirection.direction,pt.quantity,pt.procurement_Price.amount,ts.currency_Unit.name
0,2024-09-01 00:00:00+02:00,2024-09-02 00:00:00+02:00,2024-09-01 00:00:00+02:00,2024-09-01 00:15:00+02:00,PT15M,1,A51,A01,A03,A01,70,76,HUF
1,2024-09-01 00:00:00+02:00,2024-09-02 00:00:00+02:00,2024-09-01 05:00:00+02:00,2024-09-01 05:15:00+02:00,PT15M,21,A51,A01,A03,A01,70,77,HUF
2,2024-09-01 00:00:00+02:00,2024-09-02 00:00:00+02:00,2024-09-01 06:00:00+02:00,2024-09-01 06:15:00+02:00,PT15M,25,A51,A01,A03,A01,70,82,HUF
3,2024-09-01 00:00:00+02:00,2024-09-02 00:00:00+02:00,2024-09-01 07:00:00+02:00,2024-09-01 07:15:00+02:00,PT15M,29,A51,A01,A03,A01,35,6250,HUF
4,2024-09-01 00:00:00+02:00,2024-09-02 00:00:00+02:00,2024-09-01 08:00:00+02:00,2024-09-01 08:15:00+02:00,PT15M,33,A51,A01,A03,A01,31,8495,HUF


In [18]:
unique_counts = points_full_df.nunique(dropna=True).sort_values(ascending=False).to_frame("n_unique")
print(f"DataFrame shape: {unique_counts.shape}")
display(unique_counts)


DataFrame shape: (13, 1)


,n_unique
pt.procurement_Price.amount,2211
time_end_dt,696
time_dt,696
pt.quantity,166
per.start_dt,31
per.end_dt,31
pt.position,24
ts.mktPSRType.psrType,3
ts.flowDirection.direction,2
per.resolution,1


In [19]:
points_full_df_test = points_full_df[
    (points_full_df["doc.process.processType"] == "A51")
    & (points_full_df["ts.type_MarketAgreement.type"] == "A01")
    & (points_full_df["ts.mktPSRType.psrType"] == "A03")
    ] 
# aFrr and BESS & daily agreement

In [20]:
points_full_df_test.shape

(1448, 13)

In [21]:
points_full_df_test.groupby("ts.flowDirection.direction").count()

,per.start_dt,per.end_dt,time_dt,time_end_dt,per.resolution,pt.position,doc.process.processType,ts.type_MarketAgreement.type,ts.mktPSRType.psrType,pt.quantity,pt.procurement_Price.amount,ts.currency_Unit.name
ts.flowDirection.direction,,,,,,,,,,,,
A01,729,729,729,729,729,729,729,729,729,729,729,729
A02,719,719,719,719,719,719,719,719,719,719,719,719


In [22]:
points_full_df_test.sort_values("time_dt", ascending=True).head(20)

,per.start_dt,per.end_dt,time_dt,time_end_dt,per.resolution,pt.position,doc.process.processType,ts.type_MarketAgreement.type,ts.mktPSRType.psrType,ts.flowDirection.direction,pt.quantity,pt.procurement_Price.amount,ts.currency_Unit.name
0,2024-09-01 00:00:00+02:00,2024-09-02 00:00:00+02:00,2024-09-01 00:00:00+02:00,2024-09-01 00:15:00+02:00,PT15M,1,A51,A01,A03,A01,70,76,HUF
19,2024-09-01 00:00:00+02:00,2024-09-02 00:00:00+02:00,2024-09-01 00:00:00+02:00,2024-09-01 00:15:00+02:00,PT15M,1,A51,A01,A03,A02,22,200,HUF
20,2024-09-01 00:00:00+02:00,2024-09-02 00:00:00+02:00,2024-09-01 03:00:00+02:00,2024-09-01 03:15:00+02:00,PT15M,13,A51,A01,A03,A02,2,156,HUF
21,2024-09-01 00:00:00+02:00,2024-09-02 00:00:00+02:00,2024-09-01 04:00:00+02:00,2024-09-01 04:15:00+02:00,PT15M,17,A51,A01,A03,A02,22,200,HUF
1,2024-09-01 00:00:00+02:00,2024-09-02 00:00:00+02:00,2024-09-01 05:00:00+02:00,2024-09-01 05:15:00+02:00,PT15M,21,A51,A01,A03,A01,70,77,HUF
22,2024-09-01 00:00:00+02:00,2024-09-02 00:00:00+02:00,2024-09-01 06:00:00+02:00,2024-09-01 06:15:00+02:00,PT15M,25,A51,A01,A03,A02,2,159,HUF
2,2024-09-01 00:00:00+02:00,2024-09-02 00:00:00+02:00,2024-09-01 06:00:00+02:00,2024-09-01 06:15:00+02:00,PT15M,25,A51,A01,A03,A01,70,82,HUF
23,2024-09-01 00:00:00+02:00,2024-09-02 00:00:00+02:00,2024-09-01 07:00:00+02:00,2024-09-01 07:15:00+02:00,PT15M,29,A51,A01,A03,A02,21,7374,HUF
3,2024-09-01 00:00:00+02:00,2024-09-02 00:00:00+02:00,2024-09-01 07:00:00+02:00,2024-09-01 07:15:00+02:00,PT15M,29,A51,A01,A03,A01,35,6250,HUF
24,2024-09-01 00:00:00+02:00,2024-09-02 00:00:00+02:00,2024-09-01 08:00:00+02:00,2024-09-01 08:15:00+02:00,PT15M,33,A51,A01,A03,A02,15,6082,HUF


In [23]:
unique_counts = points_full_df_test.nunique(dropna=True).sort_values(ascending=False).to_frame("n_unique")
print(f"DataFrame shape: {unique_counts.shape}")
display(unique_counts)


DataFrame shape: (13, 1)


,n_unique
pt.procurement_Price.amount,1050
time_end_dt,677
time_dt,677
pt.quantity,124
per.start_dt,31
per.end_dt,31
pt.position,24
ts.flowDirection.direction,2
per.resolution,1
ts.mktPSRType.psrType,1


In [26]:
# Step 1: Extract reserve data with all relevant columns
reserve_df_raw = points_full_df_test[[
    'time_dt',
    'pt.procurement_Price.amount',
    'ts.flowDirection.direction'
]].copy()

reserve_df_raw = reserve_df_raw.rename(columns={
    'time_dt': 'datetime',
    'ts.mktPSRType.psrType': 'psr_type',
    'pt.procurement_Price.amount': 'reserve_price',
    'ts.flowDirection.direction': 'flow_direction'
})

# Convert price to numeric
reserve_df_raw['reserve_price'] = pd.to_numeric(reserve_df_raw['reserve_price'], errors='coerce')

# Ensure timezone awareness
if reserve_df_raw['datetime'].dt.tz is None:
    reserve_df_raw['datetime'] = reserve_df_raw['datetime'].dt.tz_localize('Europe/Budapest')

print(f"Raw reserve data: {len(reserve_df_raw)} records")
print(f"Date range: {reserve_df_raw['datetime'].min()} to {reserve_df_raw['datetime'].max()}")
print(f"Flow directions: {reserve_df_raw['flow_direction'].unique()}")
reserve_df_raw.head(10)


Raw reserve data: 1448 records
Date range: 2024-09-01 00:00:00+02:00 to 2024-10-01 23:00:00+02:00
Flow directions: ['A01' 'A02']


,datetime,reserve_price,flow_direction
0,2024-09-01 00:00:00+02:00,76,A01
1,2024-09-01 05:00:00+02:00,77,A01
2,2024-09-01 06:00:00+02:00,82,A01
3,2024-09-01 07:00:00+02:00,6250,A01
4,2024-09-01 08:00:00+02:00,8495,A01
5,2024-09-01 09:00:00+02:00,0,A01
6,2024-09-01 10:00:00+02:00,4678,A01
7,2024-09-01 11:00:00+02:00,0,A01
8,2024-09-01 13:00:00+02:00,7857,A01
9,2024-09-01 14:00:00+02:00,8523,A01


In [27]:
reserve_df_raw

,datetime,reserve_price,flow_direction
0,2024-09-01 00:00:00+02:00,76,A01
1,2024-09-01 05:00:00+02:00,77,A01
2,2024-09-01 06:00:00+02:00,82,A01
3,2024-09-01 07:00:00+02:00,6250,A01
4,2024-09-01 08:00:00+02:00,8495,A01
...,...,...,...
4090,2024-10-01 18:00:00+02:00,18461,A02
4091,2024-10-01 20:00:00+02:00,960,A02
4092,2024-10-01 21:00:00+02:00,758,A02
4093,2024-10-01 22:00:00+02:00,553,A02
